# Keras MNIST

## Introduction

In this kernel, we implement a Convolutional Neural Network (CNN) model with Keras using the TensorFlow backend. We use the dataset from the Digit Recognizer competition of Kaggle (see [here](https://www.kaggle.com/c/digit-recognizer)).

We will use a VGG-like architecture for our neural network, this is, it will consist of a series of convolutional, followed by fully connected layers. We will also add pooling and dropout layers to decrease the number of parameters and to regularize the model.

## Getting started

We start by importing the necessary modules:

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy.misc import toimage

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

We import the dataset into pandas dataframes

In [ ]:
df_train = pd.read_csv("../input/train.csv", encoding = 'ISO-8859-1')
df_subm =  pd.read_csv("../input/test.csv", encoding = 'ISO-8859-1')

We check that there is no missing data:

In [ ]:
df_train.isnull().sum().sum() , df_subm.isnull().sum().sum()

There is in fact, no missing data. We now take a look at the dataframes to see their structure:

In [ ]:
df_train.head()

In [ ]:
df_subm.head()

We can see that the training data consists of 785 columns, from which the first one corresponds to the label of the digit, and the other 784 correspond to all the 28x28 pixels composing each image. The submission dataframe does not contain the label, as expected. We proceed to extract the labels from the dataframe.

In [ ]:
X_train = df_train[df_train.columns[1:]]
y_train = df_train['label']

We now use the train_test_split tool from sklearn to separate our training data into two batches: one that we will use to train our neural network, and one that we will use to evaluate the scores.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)

Now we use the to_categorical tool from Keras to one-hot-encode the labels of our dataset. This is, for each possible value of the labels, we create a column, and the column corresponding to the label has a 1, while the others have a 0.

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Now that the dataset has the appropriate format, we can have a look at how the images look like:

In [ ]:
for i in range(0, 9):
    plt.subplot(330 + 1 + i)
    plt.imshow((X_train.iloc[i].values.reshape(28,28)))
plt.show()

Now we convert the dataframe into a 4 dimensional tensor. The first dimension runs through the set of examples, the second and third correspond to the two dimensions of the image and the fourth dimension correspond to the each channel of the picture (for RGB images we need three channels while for grayscale we only need one channel).

In [ ]:
X_train = X_train.values.reshape(X_train.shape[0],28,28,1)
X_test = X_test.values.reshape(X_test.shape[0],28,28,1)
# X_train = X_train.values.reshape(df_train.shape[0],28,28)

We compute now the range of values that the data takes, so we can properly normalize:

In [ ]:
X_train.max() - X_train.min() 

Now we can normalize:

In [ ]:
X_train = X_train/255
X_test = X_test/255

We are now ready to build our model.

## CNN: the sequential model

Keras makes the construction of deep neural networks much easier than Tensorflow. For this, we just need to build a sequential model, and add each layer sequentially (as the name suggests), keeping in mind that the dimensions of the output of each layer must coincide with the dimensions of the input of the next layer. 

The architecture of the CNN is based on the VGG-like convolutional neural network provided by the Keras documentation (see [here](https://keras.io/getting-started/sequential-model-guide/#examples)).

We start by building the sequential model.

In [ ]:
model = Sequential()

Now we add two groups of layers consisting of convolutions, maxpooling and dropout layers:

In [ ]:
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

We follow this by two fully connected (dense) layers, with another dropout layer.

In [ ]:
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

This finishes the architecture of the CNN. Now we choose an optimizer to train the model. We choose Stochastic Gradient Descent (SGD) with decay, momentum and Nesterov momentum for this.

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

Finally, we compile the model. We will use the categorical cross entropy as a loss function, and we will measure scores with the accuracy.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

We print a summary of the model.

In [ ]:
print(model.summary())

We are ready to train the model. We have commented this line as we will use a slightly modified version of this.

In [ ]:
# model.fit(X_train, y_train, batch_size=128, epochs=10)

We can compute the score of the model on the test dataset:

In [ ]:
# score = model.evaluate(X_test, y_test, batch_size=128)

With this model, we can obtain scores of around 0.988 on the submission data. We can improve this to get even higher scores.

## Data augmentation

In this section, we perform what is called Data Augmentation. This procedure consists of generating more data with our available dataset. The way we do this is by applying several transformations to our dataset. This is a widely used technique. For this, we will use the ImageDataGenerator tool from Keras.

We start by building the object that will randomly generate such transformations:

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    )

Here the parameters indicate that the pictures will be rotated in a random angle between -10,10 degrees, and a vertical and horizontal shift will be applied. In this way, we will count with a much more diverse dataset which will allow our model to learn better parameters for predicting in test datasets. We now fit the data generator to our training dataset.

In [ ]:
datagen.fit(X_train)

Finally, we generate the data and fit the model with the augmented data:

In [ ]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size=128),
                    steps_per_epoch=int(len(X_train) / 128), epochs=30)

And now we evaluate the model on the test data:

In [ ]:
# model.evaluate(X_test, y_test, batch_size=128)

We can check how is the model performing with the confusion matrix tool of Sklearn. This will show which numbers are being mislabeled.

In [ ]:
# confusion_matrix(np.argmax(y_test,axis=1,out=None),np.argmax(model.predict(X_test), axis=1, out=None))

We can observe that the major source of errors are 4's being labeled as 9 and vice versa. This is mostly due to how some people draw these numbers in a very similar way. Finally, we can measure the accuracy of the model with the Sklearn accuracy tool:

In [ ]:
# accuracy_score(np.argmax(y_test,axis=1,out=None),np.argmax(model.predict(X_test), axis=1, out=None))

## Submission 

Now we can proceed to generate the submission data. For this, we will use the same formatting as we used in the first part of the kernel. 

In [ ]:
#We put the values of the submission data in a matrix
test_data = df_subm.values

In [ ]:
#We reshape the matrix as a 4 dimensional tensor
test_data = test_data.reshape(test_data.shape[0],28,28,1)

In [ ]:
#We normalize the data with the same factor as we did for the train data
test_data = test_data/255

In [ ]:
#We use the model to generate the predictions
predictions = model.predict(test_data)

In [ ]:
#We get the labels of the predictions, recalling that we used a one-hot-encoding 
#to train the model
predictions = np.argmax(predictions, axis=1, out=None)

In [ ]:
#We generate a csv file with the predictions in the required format
with open("resultCNNwithPrepros.csv", "wb") as f:
    f.write(b'ImageId,Label\n')
    np.savetxt(f, np.hstack([(np.array(range(28000))+1).reshape(-1,1), predictions.astype(int).reshape(-1,1)]), fmt='%i', delimiter=",")

## Conclusion

This concludes this kernel. We believe that there is still room for improvement, either by optimizing over the architecture of the model or the hyperparameters.